In [1]:
import torch
import pyro
from pyro.optim import Adam
import congas as cg
from congas.models import MixtureCategorical
from pyro.infer import TraceEnum_ELBO


In [2]:
data = cg.simulation_data

interface = cg.Interface()

In [3]:
data["norm_factor"] = torch.sum(data["data"],axis=0) / 10**3

In [4]:
torch.mean((data["data"] / data["norm_factor"]) / 2 )

tensor(15.6250)

In [17]:
interface.set_model(MixtureCategorical)
interface.set_optimizer(Adam)
interface.set_loss(TraceEnum_ELBO)
interface.initialize_model(data)
interface.set_model_params({"lr" : 0.05,'K': 2, "nb_size_init" : torch.ones(32) * 300,
                            "theta_scale" : torch.ones(32) * 1.5,
                            "theta_rate" : torch.ones(32) * 0.1,
                           "probs" : torch.tensor([0.2,0.6,0.2,0.05,0.05])})

In [18]:
ll = interface.run(steps = 600, MAP = True)



Running MixtureCategorical on 1000 cells wiht 32 segments for 600 steps

Done!


ELBO: 105831.544000000  : 100%|██████████| 600/600 [00:45<00:00, 13.32it/s]


In [19]:
import numpy as np

lr = interface.learned_parameters()
lr

Computing assignment probabilities


{'mixture_weights': array([0.7000474 , 0.29995266], dtype=float32),
 'NB_size': array([1071.829  , 5376.039  ,  568.475  , 1863.7927 ,  451.79584,
         900.0283 , 2245.717  ,  427.8239 ,  768.7323 , 4291.236  ,
         760.19366,  846.95764, 1350.1776 , 2793.259  ,  846.86414,
         387.03128,  965.1219 ,  379.48752,  895.0243 , 1848.503  ,
        2192.0479 , 2823.3184 , 5157.5625 ,  405.18985,  280.14615,
        2872.6067 , 3904.0317 , 2091.679  ,  392.7583 ,  918.14215,
         134.65588,  421.32648], dtype=float32),
 'segment_factor': array([ 9.719708 , 13.501858 ,  8.3296585, 10.461272 ,  6.708593 ,
         9.689678 , 22.603115 , 12.555407 ,  9.585909 , 18.74698  ,
         8.804922 ,  9.548132 ,  9.707726 , 11.900031 , 10.475316 ,
         7.2867413, 10.494593 ,  7.2654324,  9.678708 , 23.321266 ,
        11.533672 , 13.378095 , 12.73171  ,  6.650995 ,  6.5486846,
        11.874583 , 12.694991 , 10.381956 ,  7.353516 ,  8.879831 ,
         4.193287 ,  7.3700657], dtype

In [20]:
import numpy as np
np.argmax(lr["assignment_probs"], axis = 0)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
print(np.argmax(lr["CNV_probabilities"][0], axis = 1) + 1)
print(np.argmax(lr["CNV_probabilities"][1], axis = 1) + 1)


[3 3 2 3 3 3 3 1 3 3 2 3 3 3 4 3 3 3 3 4 2 3 3 3 3 3 3 3 3 3 2 3]
[3 3 2 3 3 3 3 2 3 3 2 2 3 3 4 3 3 3 3 4 2 3 3 3 3 3 3 3 3 3 2 3]


In [23]:
data["pld"]

tensor([2.0000, 2.0000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 1.3000, 2.0000,
        2.0000, 1.0000, 1.7000, 2.0000, 2.0000, 3.0000, 2.0000, 2.0000, 2.0000,
        2.0000, 3.0000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000,
        2.0000, 2.0000, 2.0000, 1.0000, 2.0000])